In [1]:
import pandas as pd

# Example DataFrame
df = pd.DataFrame({
    'my_column': ['A;B', 'B;C', 'A', 'C;D', 'B', 'M;N', 'O;P', 'A;M', 'a-b; m n']
})

# Split the column by ';' into lists
df['my_column_split'] = df['my_column'].str.split(';')

# Clean values: strip, capitalize, replace spaces and hyphens with underscores
df['my_column_split'] = df['my_column_split'].apply(
    lambda lst: [x.strip().upper().replace(' ', '_').replace('-', '_') for x in lst]
)

# Explode the lists into rows
exploded = df.explode('my_column_split')

# Create binary columns for each unique value
dummies = pd.get_dummies(exploded['my_column_split'])

# Group back to original rows by max (if any value is 1, keep 1)
binary_df = dummies.groupby(exploded.index).max()

# Create business and savings flags
business_flags = {'A', 'B', 'C', 'D', 'E', 'X', 'F'}
savings_flags = {'M', 'N', 'O', 'P'}

binary_df['business'] = binary_df[list(business_flags & set(binary_df.columns))].any(axis=1).astype(int)
binary_df['savings'] = binary_df[list(savings_flags & set(binary_df.columns))].any(axis=1).astype(int)

# Concatenate with original DataFrame if needed
result = pd.concat([df, binary_df], axis=1)

result

,my_column,my_column_split,A,A_B,B,C,D,M,M_N,N,O,P,business,savings
0,A;B,"[A, B]",True,False,True,False,False,False,False,False,False,False,1,0
1,B;C,"[B, C]",False,False,True,True,False,False,False,False,False,False,1,0
2,A,[A],True,False,False,False,False,False,False,False,False,False,1,0
3,C;D,"[C, D]",False,False,False,True,True,False,False,False,False,False,1,0
4,B,[B],False,False,True,False,False,False,False,False,False,False,1,0
5,M;N,"[M, N]",False,False,False,False,False,True,False,True,False,False,0,1
6,O;P,"[O, P]",False,False,False,False,False,False,False,False,True,True,0,1
7,A;M,"[A, M]",True,False,False,False,False,True,False,False,False,False,1,1
8,a-b; m n,"[A_B, M_N]",False,True,False,False,False,False,True,False,False,False,0,0
